In [1]:
import numpy as np
import pandas as pd
from models import Hankel,Rank,Cluster,Meepc
import warnings
warnings.simplefilter('ignore')
import math
from sklearn.cluster import KMeans

In [2]:
hankel = Hankel()
rank = Rank()
cluster = Cluster()
meepc = Meepc.MEEPC()
df = pd.read_csv('~/data/training_data.csv')
df_normal=df[df['Attack LABLE (1:No Attack, -1:Attack)']==1]
df_attack=df[df['Attack LABLE (1:No Attack, -1:Attack)']==-1]
df_test=pd.read_csv('~/data/testdata.csv')
y_actual=df_test.iloc[:,-1]
labels=hankel.fit(np.array(y_actual),30,0.5)
isatt=np.any(labels<0,axis=0)
att_indices=np.where(isatt)[0]
att_indices

array([267, 268, 269, 272, 273])

In [3]:
sensors = []
for i in range(len(df_normal.columns)-1):
    X = df_normal.iloc[:,i].values
    if len(np.unique(X))>5:
        sensors.append(i)
sensors

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 55,
 56,
 58,
 59,
 60,
 61,
 62,
 63,
 72,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96]

In [4]:
def find_threshold(radii_n,radii_a):
    label = [1]*len(radii_n) + [-1]*len(radii_a)
    label = np.array(label)
    radiis = np.array(list(radii_n)+list(radii_a))
    indices = np.argsort(radiis)
    label = label[indices]
    pos_temp = (1+label)//2
    neg_temp = (1-label)//2
    tp = np.cumsum(pos_temp)
    fp = np.cumsum(neg_temp)
    fn = tp[-1] - tp
    fmeas = 2*tp / (2*tp + fp + fn)
    # print(fmeas)
    idx = np.argmax(fmeas)
    # print(indices[idx])
    return radiis[indices[idx]]

In [5]:
# def calc_radius(X,optimal_k)

In [6]:
for sens in sensors:
    X = df_normal.iloc[:,sens].values
    X = hankel.fit(X,30,0.5)
    X=X.T 
    kmeans,optimal_k = cluster.fit(X)
    kmeans.fit(X)
    # print(optimal_k)
    radiis_normal = []
    weights=[]
    centers=[]
    clusters_R=[]
    clusters_V=[]
    for i in range(optimal_k):
        cluster_ = X[np.where(kmeans.labels_ == i)[0]]
        r = rank.fit(cluster_)
        clusters_R.append(r)
        # print("r: "+str(r))
        U,Sigma,VT = np.linalg.svd(cluster_)
        V = VT.T
        clusters_V.append(V)
        cluster_ = np.matmul(cluster_,V[:,:r])
        weight,center = meepc.fit(cluster_)
        weights.append(weight)
        centers.append(center)
        var1=np.square(cluster_-center)
        var2=np.matmul(weight,var1.T)
        radiis_normal.append(np.sqrt(var2))
   
    X_attack = df_attack.iloc[:,sens].values
    X_att = hankel.fit(X_attack,30,0.5)
    X_att=X_att.T 
    radiis_attack = []
    for i in range(optimal_k):
        cluster_ = np.matmul(X_att,clusters_V[i][:,:clusters_R[i]])
        var1=np.square(cluster_-centers[i])
        var2=np.matmul(weights[i],var1.T)
        radiis_attack.append(np.sqrt(var2))
    threshold_clusters = [0]*optimal_k
    for i in range(optimal_k):
        threshold_clusters[i] = find_threshold(radiis_normal[i],radiis_attack[i])

    X_test=df_test.iloc[:,sens].values
    Xtest=hankel.fit(X_test,30,0.5)  #size -> 1800x274
    Xtest=Xtest.T  
    distances=[]
    for i in range(optimal_k):
        Xtest_cluster=np.matmul(Xtest,clusters_V[i][:,:clusters_R[i]]) #size -> 274 x R
        var1=np.square(Xtest_cluster-centers[i])
        var2=np.matmul(weights[i],var1.T)
        distance=np.sqrt(var2)
        distances.append(distance)

    stacked_arr=np.vstack(distances)
    distances_arr=np.transpose(stacked_arr)
    check_anamoly=threshold_clusters-distances_arr     # positive = inside cluster | negative = outside cluster
    #check for if any datapoint is coming inside the cluster
    is_positive=np.any(check_anamoly>0,axis=1)
    #indices of those rows
    positive_indices=np.where(is_positive)[0]
    for i in [[267, 268, 269, 272, 273]]:
        print(distances_arr[i])
    for i in range(274):
        if i not in positive_indices:
            print(i,end=' ')
    print()

[[1.21963236 0.98856793 1.81665352 2.10199312 0.96211073 1.71467513
  1.90176519 0.88226566]
 [1.53565423 1.17676397 1.29062965 2.46250033 0.57265041 1.33098733
  2.68349643 1.62078409]
 [1.71340001 1.18839224 1.2784942  2.47430102 0.56014227 1.32670617
  2.7037151  1.64769355]
 [1.82207934 0.860128   1.43480175 2.15328544 0.66627307 1.31150494
  2.20069066 1.84200652]
 [1.42204467 1.20946683 1.39506149 2.43557163 0.5981568  1.46451
  2.60166152 1.88550455]]
64 127 131 159 189 199 219 243 246 247 255 
[[0.5074731  0.58102943 0.        ]
 [0.45801657 0.47355969 0.        ]
 [0.49208625 0.44713326 0.        ]
 [0.54801171 0.50825504 0.        ]
 [0.50805295 0.68859344 0.        ]]

[[0.73678902 1.0825443 ]
 [0.56881213 1.58501638]
 [0.56738215 1.57610078]
 [0.57649337 1.64031802]
 [0.61990805 1.81069923]]

[[1.35847293 1.00356844]
 [1.5123847  0.59586809]
 [1.55061078 0.61445236]
 [0.7167533  0.61276806]
 [1.07239416 0.65730577]]
243 
[[0.80280879 0.70951226]
 [0.9052708  0.36545264]
 [0